# Potts Model

In [1]:
from PottsModel import PottsModel

import numpy as np
import matplotlib.pyplot as plt

/Users/bertdepoorter/Nextcloud/Documents/School/Universiteit/Monte Carlo Methods/PottsModel.py:120: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [i, int((j+1+N)%N)]       # right neighbour


## Functionality
The Potts Model class has the following functions available:
- initialize spins: create a lattice with spins randomly assigned
- calculate_energy: calculate the energy of a given spin configuration
- get_neighbours: get the indices of the neighbouring entries under specific boundary conditions
- sample_spin_configurations: sample a given number of spin configurations for a specified sampling method
  (Options are 'heat bath' and 'metropolis')
- get_magnetization: calculate total magnetization for a given spin configuration
- plot_magnetization: create plot similar to figure 3 in lecture notes for a specified number of chains.


In [2]:
# specify parameters
N = 10
num_samples = 70000

In [3]:
# initialize model with het bath sampling
model_heat_bath = PottsModel(
    N,
    10, 
    sampling_method='heat bath'
)

In [4]:
# run sampling
chain_heat_bath = model_heat_bath.sample_spin_configurations(num_samples)

In [5]:
# Initialize model with metropolis smapling
model_metropolis = PottsModel(
    N, 
    10,
    sampling_method='metropolis'
)

In [ ]:
# run sampling
chain_metropolis=model_metropolis.sample_spin_configurations(num_samples)

In [ ]:
# Make plot of both sampling methods

# create array of chains
spin_samples = np.asarray([chain_heat_bath, chain_metropolis])

# Create plot
model_metropolis.plot_magnetizations(
    num_samples, 
    chains=2, 
    spin_samples=spin_samples, 
    chain_labels=['Heat bath sampling', 'Metropolis sampling']
)

In [ ]:
M_metropolis = model_metropolis.magnetization_chain(chain_metropolis)
M_avg_metropolis = np.sum(M_metropolis)/len(M_metropolis)
print(M_avg_metropolis)

In [ ]:
M_heat_bath = model_heat_bath.magnetization_chain(chain_heat_bath)
M_avg_heat_bath = np.sum(M_heat_bath)/len(M_heat_bath)
print(M_avg_heat_bath)